# Scrubbing User Data
We want to ensure that the data that we analyze is scrubbed so that no trace-able information is related to individual data points.

Import Dependencies

In [6]:
# import dependencies
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sci
import pandas as pd

Load our raw data in the form of `CSV` into data frames. Raw data should be located in `./raw-data/`.

In [7]:
file_name = input()
file_path = "./raw-data/" + file_name
raw_data = pd.read_csv(file_path)